In [1]:
import pandas as pd
import ipywidgets as widgets

In [2]:
from ipywidgets import AppLayout

# this is just for the header button
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, 
                          layout=widgets.Layout(height='auto', width='auto'))

# button for top of scoring table.  Could have used other widgets...
header_button = create_expanded_button("Clifton Team Scoring", 'success')

# define arrangement of label and +/- buttons for individual box:
def interButtonLabel(text = "test", value = 0):
    strFormat = "{}: <b>{}</b>"
    strText = strFormat.format(text, value)
    name_layout = widgets.Layout(width = "100px", max_width = "150px")
    label = widgets.HTML(strText)
    label.max_width = "10px"
    label.nvalue = value
    box_layout = widgets.Layout(display='flex',
            flex_flow='column',
            align_items='center',
            width='50%',
            margin = "0px 0px 0px 10px")
    up = widgets.Button(description ="+", button_style = 'info', layout = box_layout)
    down = widgets.Button(description = "-", button_style = 'danger', layout = box_layout)
    bv = widgets.VBox([up, down])
    bh = widgets.HBox([label, bv], layout = widgets.Layout(border = '1px solid black'))
    bh.index = text
    bh.value = value
        
    def adjust(b):
        change = 1 if b.description=='+' else -1
        label.nvalue += change
        bh.value += change
        label.value = strFormat.format(text, label.nvalue)
       
    up.on_click(adjust)
    down.on_click(adjust)   
    return bh

# define what each row looks like as combination of name and list of interButtonLabels
itemList = ['2PM', '2PA', '3PM', '3PA', 'FTM', 'FTA', 'OR', 'DR', 'A', 'S', 'Blk', 'TO', 'F']
def nameLine(name = "Dumars, Joe", items = itemList):
    nameLay = widgets.Layout(width = '100px')
    nameWidg = widgets.HTML(f"<b><font color='blue'><font size=2>{name}</b>", layout = nameLay)
    widgList = [interButtonLabel(item) for item in items]
    
    # super clumsy way of connecting buttons withing a line....
    def secondary_2PA_adjust(b):
        change = 1 if b.description=='+' else -1
        widget2 = widgList[1]
        label = widget2.children[0]
        label.nvalue += change
        widget2.value += change
        label.value = "{}: <b>{}</b>".format(widget2.index, label.nvalue)

    def secondary_3PA_adjust(b):
        change = 1 if b.description=='+' else -1
        widget2 = widgList[3]
        label = widget2.children[0]
        label.nvalue += change
        widget2.value += change
        label.value = "{}: <b>{}</b>".format(widget2.index, label.nvalue)
    
    def secondary_FTA_adjust(b):
        change = 1 if b.description=='+' else -1
        widget2 = widgList[5]
        label = widget2.children[0]
        label.nvalue += change
        widget2.value += change
        label.value = "{}: <b>{}</b>".format(widget2.index, label.nvalue)
        
    widgList[0].children[1].children[0].on_click(secondary_2PA_adjust)
    widgList[0].children[1].children[1].on_click(secondary_2PA_adjust)
    widgList[2].children[1].children[0].on_click(secondary_3PA_adjust)
    widgList[2].children[1].children[1].on_click(secondary_3PA_adjust)
    widgList[4].children[1].children[0].on_click(secondary_FTA_adjust)
    widgList[4].children[1].children[1].on_click(secondary_FTA_adjust)
    hOut = widgets.HBox([nameWidg] + widgList)
    hOut.name = name
    return hOut

In [3]:
roster = ["Adrian", "Bate", "Brennan", "Collin", "Connor", "Dallin", "Derrek", "Gunnar", "Miles", "Treyton", "Zach"]
outbox = widgets.Output(layout = {'border': '1px solid black'})
# initialize dictionary of results:
out = [nameLine(name) for name in sorted(roster)]
vOut = widgets.VBox(out)
tableTrans = {'2PM':'2PM', '2PA':'2PA', '3PM':'3PM', '3PA':'3PA', 'FTM':'FTM',
              'FTA':'FTA', 'OR':'O Reb', 'DR':'D Reb', 'A':'Assists', 'S':'Steals',
              'Blk':'Blocks', 'TO':'Turnovers', 'F':'Fouls'}

def update_table(b):
    # clear data and table
    data = {item: [] for item in itemList}
    outbox.clear_output()
    # rebuild data and display
    for line in vOut.children:
        for box in line.children[1:]:
            data[box.index].append(box.value)
    df = pd.DataFrame(data, index = sorted(roster)).rename(columns=tableTrans)
    styles = [dict(selector="caption", 
                props=[("text-align", "center"),
                ("font-size", "150%"),
                ("color", 'black')])]    # the color value can not be None
    with outbox:
        out = df.style \
                .set_caption("Summary Table") \
                .set_table_styles(styles) \
                .set_properties(**{'text-align':'center'})
        display(out)
        
for line in vOut.children:
    for box in line.children[1:]:
        for button in box.children[1].children:
            button.on_click(update_table)
            
widgets.AppLayout(header=header_button,
          center = vOut,
          footer = outbox,
          pane_heights = [1, 12, 6]
         )

AppLayout(children=(Button(button_style='success', description='Clifton Team Scoring', layout=Layout(grid_area…